In [54]:
from glob import glob
import yaml
import os
from IPython.display import Image

In [55]:
## variables

label_num = '1'

# 826/846
images_path = 'C:\\Users\\2020\\Desktop\\labeling workspace\\1_spray\\'
labels_path = 'C:\\Users\\2020\\Desktop\\labeling workspace\\1_spray_label\\'

# images_path = 'C:\\Users\\2020\\Desktop\\labeling workspace\\2_portable_charger_total_test\\'
# labels_path = 'C:\\Users\\2020\\Desktop\\labeling workspace\\2_portable_charger_total_label_2_test\\'

In [56]:
## function

#6
def classes_checker(labels_list):
    global images_path, labels_path
    if labels_path + 'classes.txt' in labels_list:
        print('classes.txt exists')
        return [labels_path + 'classes.txt']
    else:
        return []

#0 for empty_checker
def readline_txt_to_list(label_path):
    list_temp = []
    with open(label_path, 'r') as f:
        while True:
            line = f.readline()
            if line == '':
                return list_temp
            else:
                list_temp.append(line)

#1
def empty_checker(labels_list):
    empty_label_list = []
    for label_path in labels_list:
        if readline_txt_to_list(label_path) == []:
            print(label_path + ' is empty')
            empty_label_list.append(label_path)
    if empty_label_list == []:
        print('no empty files')            
    return empty_label_list

#2
def label_num_checker(labels_list):
    global label_num
    wrong_num_list = []
    for label_path in labels_list:
        with open(label_path, 'r') as f:
            while True:
                line = f.readline()
                if line == '':
                    break
                elif line[0] != label_num:
                    print('there exists wrong label_num(' \
                        + line[0] + ') in ' + label_path)
                    wrong_num_list.append(label_path)
    return wrong_num_list

#4
def file_extension_checker(images_list, labels_list):
    wrong_ext_images = []
    wrong_ext_labels = []    
    for image_path in images_list:        
        if image_path[-4:] != '.jpg':
            print(image_path + ' has wrong file extension')
            wrong_ext_images.append(image_path)
    for label_path in labels_list:
        list_temp = []
        if label_path[-4:] != '.txt':
            print(label_path + ' has wrong file extension')
            wrong_ext_labels.append(label_path)
    return wrong_ext_images, wrong_ext_labels

#3 before use this, remove classes.txt by classes_checker()
#3 + assume, there are only .jpg and .txt
def bijection_checker(images_list, labels_list):
    global images_path, labels_path
    residual_image_list = []
    residual_label_list = []
    for image_path in images_list:
        # crspd= corresponding
        crspd_label_path = labels_path + \
            image_path.replace(images_path, '')[:-4] + '.txt'
        if not crspd_label_path in labels_list:
            residual_image_list.append(image_path)
    for label_path in labels_list:
        crspd_image_path = images_path + \
            label_path.replace(labels_path, '')[:-4] + '.jpg'
        if not crspd_image_path in images_list:
            residual_label_list.append(label_path)
    if residual_image_list + residual_label_list != []:
        print('images and labels are not 1:1')
        print('residual label :\n', residual_label_list)
        print('residual image :\n', residual_image_list)
    else:
        print('images and labels are 1:1')
    return residual_image_list, residual_label_list

#5
def sample_checker(images_list, labels_list):
    global images_path, labels_path
    sample_list = ['00.knife_naver_001',
                   '01.spray_naver_001',
                   '02.portable_charger_naver_001',
                   '03.lighter_naver_001',
                   '04.e_cigarette_naver_001'
                   ]
    sample_image_list = []
    sample_label_list = []
    for i in range(len(sample_list)):
        sample_image_path = images_path + sample_list[i] + '.jpg'
        if sample_image_path in images_list:
            print('sample\n' + sample_image_path + '\nis in images')
            sample_image_list.append(sample_image_path)
    for i in range(len(sample_list)):
        sample_label_path = labels_path + sample_list[i] + '.txt'
        if sample_label_path in labels_list:
            print('sample\n' + sample_label_path + '\nis in labels')
            sample_label_list.append(sample_label_path)
    return sample_image_list, sample_label_list

def del_images_and_labels(del_image_list, del_label_list):
    for image_path in del_image_list:
        if os.path.isfile(image_path):
            os.remove(image_path)
    for label_path in del_label_list:
        if os.path.isfile(label_path):
            os.remove(label_path)

def get_lists():
    global images_path, labels_path
    images_list = glob(images_path + '*')
    labels_list = glob(labels_path + '*')
    if images_path + 'desktop.ini' in images_list:
        images_list.remove(images_path + 'desktop.ini')
    if labels_path + 'desktop.ini' in labels_list:
        labels_list.remove(labels_path + 'desktop.ini')
    return images_list, labels_list

# after empty_checker, you can use it
def relabeler(label_num, labels_list):
    for label_path in labels_list:
        new_lines = ''
        with open(label_path, 'r') as f:
            lines = f.readlines()
            for line in lines:
                new_lines += line.replace(line.split(' ')[0], label_num, 1)
                print(label_path + ' is relabeled by ' + label_num)
        with open(label_path, 'w') as f:
            f.write(new_lines)
    
def renamer(): # no duplication
    pass

def main():
    images_list, labels_list = get_lists()
    del_images_and_labels([], classes_checker(labels_list)) #6
    images_list, labels_list = get_lists()
    del_images_and_labels(*sample_checker(images_list, labels_list)) #5
    images_list, labels_list = get_lists()
    del_images_and_labels(*file_extension_checker(images_list, labels_list)) #4
    images_list, labels_list = get_lists()
    del_images_and_labels([], empty_checker(labels_list)) #1
    images_list, labels_list = get_lists()
    del_images_and_labels(*bijection_checker(images_list, labels_list)) #3
    # images_list, labels_list = get_lists()
    # del_images_and_labels([], label_num_checker(labels_list)) #2
    images_list, labels_list = get_lists()
    relabeler(label_num, labels_list)
    
    images_list, labels_list = get_lists()
    print('num of images :', len(images_list), '\nnum of labels :', len(labels_list))
    
    ######################
    # Now, check and del #
    ######################


In [ ]:
line1 = '7 125423 24'
line

In [36]:
emptylist = ['', '', '']
emptylist[0]

''

In [35]:
str1 = ' '
str2 = ''
print(str1.strip())
print(str2.strip())
print(str2)
print(type(str1.strip()))
print(type(str2.strip()))
print(type(str2))
print('1' + str1 + '1')
print(str1.strip() == str2)




<class 'str'>
<class 'str'>
<class 'str'>
1 1
True


In [57]:
## main

if __name__ == '__main__':
    main()

no empty files
images and labels are 1:1
C:\Users\2020\Desktop\labeling workspace\1_spray_label\1.txt is relabeled by 1
C:\Users\2020\Desktop\labeling workspace\1_spray_label\10.txt is relabeled by 1
C:\Users\2020\Desktop\labeling workspace\1_spray_label\101.txt is relabeled by 1
C:\Users\2020\Desktop\labeling workspace\1_spray_label\102.txt is relabeled by 1
C:\Users\2020\Desktop\labeling workspace\1_spray_label\103.txt is relabeled by 1
C:\Users\2020\Desktop\labeling workspace\1_spray_label\104.txt is relabeled by 1
C:\Users\2020\Desktop\labeling workspace\1_spray_label\106.txt is relabeled by 1
C:\Users\2020\Desktop\labeling workspace\1_spray_label\107.txt is relabeled by 1
C:\Users\2020\Desktop\labeling workspace\1_spray_label\108.txt is relabeled by 1
C:\Users\2020\Desktop\labeling workspace\1_spray_label\109.txt is relabeled by 1
C:\Users\2020\Desktop\labeling workspace\1_spray_label\110.txt is relabeled by 1
C:\Users\2020\Desktop\labeling workspace\1_spray_label\111.txt is relab

In [16]:
print(len(images_list))
images_list

336


['C:\\Users\\2020\\Desktop\\labeling workspace\\2_portable_charger_total_test\\02.portable_charger_naver_001.jpg',
 'C:\\Users\\2020\\Desktop\\labeling workspace\\2_portable_charger_total_test\\img_portable_charger_total0001.jpg',
 'C:\\Users\\2020\\Desktop\\labeling workspace\\2_portable_charger_total_test\\img_portable_charger_total0009.jpg',
 'C:\\Users\\2020\\Desktop\\labeling workspace\\2_portable_charger_total_test\\img_portable_charger_total0020.jpg',
 'C:\\Users\\2020\\Desktop\\labeling workspace\\2_portable_charger_total_test\\img_portable_charger_total0038.jpg',
 'C:\\Users\\2020\\Desktop\\labeling workspace\\2_portable_charger_total_test\\img_portable_charger_total0043.jpg',
 'C:\\Users\\2020\\Desktop\\labeling workspace\\2_portable_charger_total_test\\img_portable_charger_total0051.jpg',
 'C:\\Users\\2020\\Desktop\\labeling workspace\\2_portable_charger_total_test\\img_portable_charger_total0054.jpg',
 'C:\\Users\\2020\\Desktop\\labeling workspace\\2_portable_charger_total_

In [17]:
print(len(labels_list))
labels_list

337


['C:\\Users\\2020\\Desktop\\labeling workspace\\2_portable_charger_total_label_2_test\\02.portable_charger_naver_001.txt',
 'C:\\Users\\2020\\Desktop\\labeling workspace\\2_portable_charger_total_label_2_test\\classes.txt',
 'C:\\Users\\2020\\Desktop\\labeling workspace\\2_portable_charger_total_label_2_test\\img_portable_charger_total0001.txt',
 'C:\\Users\\2020\\Desktop\\labeling workspace\\2_portable_charger_total_label_2_test\\img_portable_charger_total0009.txt',
 'C:\\Users\\2020\\Desktop\\labeling workspace\\2_portable_charger_total_label_2_test\\img_portable_charger_total0020.txt',
 'C:\\Users\\2020\\Desktop\\labeling workspace\\2_portable_charger_total_label_2_test\\img_portable_charger_total0038.txt',
 'C:\\Users\\2020\\Desktop\\labeling workspace\\2_portable_charger_total_label_2_test\\img_portable_charger_total0043.txt',
 'C:\\Users\\2020\\Desktop\\labeling workspace\\2_portable_charger_total_label_2_test\\img_portable_charger_total0051.txt',
 'C:\\Users\\2020\\Desktop\\labe

label_num = 1
image_path = "~~~"
label_path = "~~~"

1. `.txt` 중에 빈 파일이 있는지 체크해서 print 하고, 빈파일명 list를 반환 return

2. 모든 `.txt`에 레이블링 번호가 label_num과 다른게 있는지 체크. - labling number error print하고

   레이블링 번호 안맞는 파일명 list 반환

3. `.jpg`와 `.txt`가 1:1 mapping 되는지 체크 (단순히 갯수만 체크하는거 말고, 더 정확하게) - integrity error

4. `.jpeg`, `.png` 등 확장자가 다른게 있는지 체크 - filename extension error

5. 앞이 00, 01, 02, 03, 04로 시작하는 파일(샘플 이미지 파일)이 여러개 있는지 체크 - sample duplication error

6. def del_image(list):, def del_text(list):

?. classes 로 시작하는 파일이 있는지 체크 - classes duplication error

+@ 용량이 같은 이미지가 있다면 출력? (용량이 같으면 같은이미지일 확률이 몇%나 될지? test)

​	file size duplication error => image duplication error ??